In [1]:
import os
import numpy as np
import torch
torch.set_num_threads(1)
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import argparse
from tqdm import tqdm
import logging
from model import TransformerVAE
from utils import AverageMeter
from dataset import get_dataloader
from model.losses import VAELoss, div_loss
from dataset import ReactionDataset


In [2]:
import torch

torch.multiprocessing.set_start_method('spawn')


In [3]:
# import dill as pickle
from dataset import ReactionDataset
from torch.utils.data import DataLoader
from model import TransformerVAE
import pdb
from tqdm import tqdm
# from model import HuBERTEncoder


print ('num threads: ', torch.get_num_threads())
# torch.set_num_threads(1)
audio_flag = True
model = TransformerVAE(
    img_size=256,    
    use_hubert=audio_flag,
audio_dim=128, # becomes irrelevant in case for hubert
max_seq_len=751, 
seq_len=750)

dataset = ReactionDataset('../data', 'data/sample_udiva.csv', clip_length=750, use_raw_audio=audio_flag, load_3dmm_l=True, mode='val')
shuffle = True 
# sample = dataset[0]
# hubert = HuBERTEncoder(1024)
# hubert = hubert.cuda()

train_loader = DataLoader(dataset=dataset, batch_size=2, shuffle=shuffle, num_workers=2)

checkpoints = torch.load('./results/marlin_hubert/best_checkpoint.pth', map_location=torch.device('cpu'))
state_dict = checkpoints['state_dict']
model.load_state_dict(state_dict)
model = model.cuda()
model.eval()
# speaker_video_clip, speaker_audio_clip, _, _, _, _, listener_emotion, listener_3dmm, listener_references = dataset[0]
# print (listener_3dmm.shape)
# print ("enumeratin")
for batch_idx, (speaker_video_clip, 
                speaker_video_orig, 
                speaker_audio_clip, speaker_emotion, _, 
                listener_video_clip, _, listener_emotion, listener_3dmm, listener_references) in enumerate(tqdm(train_loader)):

    print ('speaker video shape: ', speaker_video_clip.shape)
    print ('speaker audio shape: ', speaker_audio_clip.shape)
    print ('listener 3d mm shape: ', listener_3dmm.shape)
    print ("running inference...")
    
    with torch.no_grad():
        listener_3dmm_out, listener_emotion_out, distribution = model(speaker_video_clip.cuda(), speaker_audio_clip.cuda())

print ("done.")
# print ('Batch idx: ', batch_idx)
# train_loader = get_dataloader(args, "data/train.csv", load_ref=False, load_video_l=False)

num threads:  1
Using HuBERT.


  0%|                                                                                                                                                   | 0/1 [00:00<?, ?it/s]

speaker video shape:  torch.Size([2, 1568, 1024])
speaker audio shape:  torch.Size([2, 480000])
listener 3d mm shape:  torch.Size([2, 750, 58])
running inference...


/home/surbhi/anaconda3/envs/react_new/lib/python3.8/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343962757/work/aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:14<00:00, 14.58s/it]

done.


In [4]:
# render = Render()

tensor([[-0.0571,  0.1654,  0.0706,  ...,  0.0467, -0.0047,  0.0558],
        [-0.0571,  0.1654,  0.0706,  ...,  0.0467, -0.0047,  0.0558],
        [-0.0571,  0.1654,  0.0706,  ...,  0.0467, -0.0047,  0.0558],
        ...,
        [-0.0571,  0.1654,  0.0706,  ...,  0.0467, -0.0047,  0.0558],
        [-0.0571,  0.1654,  0.0706,  ...,  0.0467, -0.0047,  0.0558],
        [-0.0571,  0.1654,  0.0706,  ...,  0.0467, -0.0047,  0.0558]],
       device='cuda:0')